In [2]:
%load_ext autoreload
%autoreload 2



In [3]:
from runner import *
from process_results import *
import os
import pandas as pd

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:

data = {
    "name": [],
    "num_exce": [],
    "num_frame": [],
    "is_async": [],
    "logged": [],
    "dynamic": [],
    "static": [],
    "hybrid_online": [],
    "hybrid_offline": [],
    "distance": [], 
    "running_service": []
}

for name, app in TEST_APPLICATIONS.items():
    data["name"].append(name)
    ground_truth = app.read_ground_truth()
    dynamic_dependency = app.read_latest_dynamic_dependency()
    root_cause_in_log = check_root_cause_in_log(ground_truth, app.origin_log_path)
    data["is_async"].append(app.is_async)
    data["logged"].append(root_cause_in_log)

    (tp, fp, fn) = process_dependency_result(dynamic_dependency, ground_truth)
    data["dynamic"].append(f"{tp}/{fp}/{fn}")

    static_dependency = app.read_latest_static_dependency()
    (tp, fp, fn) = process_dependency_result(static_dependency, ground_truth)
    data["static"].append(f"{tp}/{fp}/{fn}")

    hybrid_online, hybrid_offline = app.read_latest_hybrid_dependency()
    (tp, fp, fn) = process_dependency_result(hybrid_online, ground_truth)
    data["hybrid_online"].append(f"{tp}/{fp}/{fn}")
    (tp, fp, fn) = process_dependency_result(hybrid_offline, ground_truth)
    data["hybrid_offline"].append(f"{tp}/{fp}/{fn}")
    data["distance"].append(get_exception_distance(ground_truth,
                                                   os.path.join(app.get_latest_result("dynamic"), "exception.json")))


    dynamic_result_path = app.get_latest_result("dynamic")
    exceptions = read_exceptions(os.path.join(dynamic_result_path, "exception.json"))
    data['num_exce'].append(len(exceptions))
    data["num_frame"].append(len(open(os.path.join(app.get_latest_result(
        "dynamic"), "affected-var-results.json")).readlines()))

    data['running_service'].append(app.is_running_service)

df = pd.DataFrame(data)
print(df)

FileNotFoundError: [Errno 2] No such file or directory: '/usr0/home/aoli/repos/exchain/applications/scripts/data/wicket-6908.json'

In [10]:

data = {
    "name": [],
    "type": [],
    "origin": [],
    "static": [],
    "hybrid": [], 
    "dynamic": []
}
results = []
for name, app in BENCH_APPLICATIONS.items():
    separate_results = ["fineract_bench", "jena_bench"]
    print(name)
    if "jena" in name:
        continue
    results.append(read_perf_result(app))
df = pd.DataFrame(results)
df.columns = ["name", "origin", "std", "static", "std", "diff", "hybrid", "std", "diff", "dynamic", "std", "diff"]
print(df)

#     origin_perf_result = app.perf_result_path("origin")
#     static_perf_result = app.perf_result_path("static")
#     hybrid_perf_result = app.perf_result_path("hybrid")
#     dynamic_perf_result = app.perf_result_path("dynamic")

#     if name in separate_results:
#         result = read_separate_perf_result([origin_perf_result, static_perf_result, hybrid_perf_result, dynamic_perf_result])
#     else:
#         result = read_aggregate_perf_result([origin_perf_result, static_perf_result, hybrid_perf_result, dynamic_perf_result])
    
#     for key, value in result.items():
#         if key == "tpr":
#             m = "ms"
#         if key == "throughput":
#             m = "op/s"
#             key = "tp"
#         if key == "exec_time":
#             m = "s"
#             key = "time"
#         data["name"].append(name)
#         data["type"].append(key + f"({m})")
#         data["origin"].append(value[0])
#         data["static"].append(value[1])
#         data["hybrid"].append(value[2])
#         data["dynamic"].append(value[3])
# df = pd.DataFrame(data)
# print(df)
# uniformed benchmark?
# sosp 19 crash tuner
# https://dl.acm.org/doi/pdf/10.1145/3341301.3359645
# spectrum based fault localization
    # statistical debugging
# https://www.usenix.org/system/files/osdi18-cui.pdf

# show that first request is benign

# we propose this formulation
# main contribution is: 

# we propose to use taint analysis to chain the variables together

# explore the design space
# different apprach for different scenario



fineract_bench
jena_bench
hadoop_nnbench
wicket_bench
solr_bench
hadoop_terasort
tomcat_bench
hadoop_dfs_write
               name   origin    std   static    std   diff   hybrid   std  \
0    fineract_bench     12.8    0.1     13.7    0.1   7.1%     16.1   0.1   
1    hadoop_nnbench   1542.0    7.0   1528.2    6.3  -0.9%   1216.1   2.7   
2      wicket_bench     26.7    1.2     27.8    1.1   4.1%     31.0   0.7   
3        solr_bench  16623.6  111.2  16095.6  105.6  -3.2%  12062.7  86.4   
4   hadoop_terasort     20.6    0.0     22.7    0.1  10.0%     26.9   0.3   
5      tomcat_bench     15.3    2.8     15.5    0.9   1.3%     17.3   2.1   
6  hadoop_dfs_write   2136.5   31.8   2125.5   27.4  -0.5%   1810.3  14.3   

     diff dynamic   std     diff  
0   25.3%   205.4  11.8  1503.0%  
1  -21.1%   690.8   2.0   -55.2%  
2   16.2%   457.8  16.3  1613.5%  
3  -27.4%   220.2  29.1   -98.7%  
4   30.4%  1231.8  32.5  5871.5%  
5   13.1%   243.9   3.9  1496.8%  
6  -15.3%   316.5  16.6   -